 🤖 My First Groq-Powered Coding Assistant (Notebook Edition)

Welcome! This Jupyter Notebook will guide you through creating a simple coding assistant that uses the Groq API for fast responses from Large Language Models (LLMs) like Llama 3.

**What you'll learn:**
- How to install and import necessary Python libraries.
- How to securely configure your Groq API key.
- How to connect to the Groq service.
- How to send prompts to an LLM and get responses.
- How to use "tools" (like getting the current time) with the LLM.
- How to evaluate the LLM's responses.
- How to create a basic interactive chat loop.

Let's get started!

## ⚙️ Step 0: Setup - Install Libraries

First, we need to make sure we have the `groq` library (for interacting with the Groq API) and `python-dotenv` (for managing our API key securely).

**Instructions:**
1. Run the code cell below **once**.
2. If the libraries are already installed, it won't do any harm.
3. After running it successfully, you can comment out the `!pip install` lines (by adding a `#` at the beginning of those lines) to avoid running them every time you open the notebook.

In [ ]:
# Before running this, make sure you have Python installed.
# You can run these commands in your terminal or directly in a notebook cell by adding "!" at the beginning.
# If you run them here, they only need to be run once.

# print("Installing necessary libraries...")
# !pip install groq python-dotenv

# print("Installation complete! You can now comment out the !pip install lines or remove this cell.")

Now, let's import the libraries we'll need for this project.

In [1]:
import os
import json
from groq import Groq, RateLimitError, APIError
from datetime import datetime
from dotenv import load_dotenv # For loading API key from a .env file

print("Libraries imported successfully!")

Libraries imported successfully!


## 🔑 Step 1: Configure Your Groq API Key

To use the Groq API, you need an API key. You can get one from the [Groq Console](https://console.groq.com/keys).


**For Local Jupyter:**
1.  **Create a file named `.env`** in the *same directory* as this notebook.
2.  Open the `.env` file with a text editor.
3.  Add the following line, replacing `your_gsk_key_here` with your actual Groq API key:
    ```
    GROQ_API_KEY="your_gsk_key_here"
    ```
4.  Save the `.env` file.

**For Google Colab (Preferred for Colab):**
1. Click the **key icon (🔑)** in the left sidebar of Colab.
2. Click `+ ADD A NEW SECRET`.
3. Name: `GROQ_API_KEY`
4. Value: Your actual Groq API key.
5. Ensure 'Notebook access' is ON.
The code in the next cell is set up for local `.env` but will need adjustment for Colab secrets (see Colab instructions I provided earlier).

In [2]:
# Load environment variables from .env file (for local use)
# For Colab, you'd use: from google.colab import userdata; GROQ_API_KEY = userdata.get('GROQ_API_KEY')
load_dotenv()

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

if not GROQ_API_KEY:
    print("GROQ_API_KEY not found. Ensure it's in .env (local) or Colab Secrets and you've adapted this cell if in Colab.")
    # Fallback for manual input if needed, but not recommended for routine use:
    # GROQ_API_KEY = input("Please enter your Groq API Key: ")
    if not GROQ_API_KEY: # Check again if it was entered
        raise ValueError(
            "Groq API Key is not set. Please configure it and restart the kernel."
        )
else:
    print("Groq API Key loaded (or attempted from environment/Colab Secrets).")

# --- Configuration ---
DEFAULT_MODEL = "llama3-8b-8192"
EVALUATION_MODEL = "llama3-8b-8192" # Can be the same or different

print(f"Using model: {DEFAULT_MODEL}")
print(f"Using evaluation model: {EVALUATION_MODEL}")

Groq API Key loaded (or attempted from environment/Colab Secrets).
Using model: llama3-8b-8192
Using evaluation model: llama3-8b-8192


## 🔗 Step 2: Connect to the Groq Client

Now that we have the API key, let's write a function to initialize the Groq client. This client is what we'll use to communicate with the Groq API.

In [3]:
def get_groq_client():
    """Initializes and returns the Groq client."""
    if not GROQ_API_KEY:
        print(
            "Groq API Key not found. Please ensure it's set."
        )
        return None
    try:
        client = Groq(api_key=GROQ_API_KEY)
        print("Groq client initialized successfully.")
        return client
    except APIError as e:
        print(f"Failed to initialize Groq client due to APIError: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during client initialization: {e}")
        return None

# Attempt to initialize the client
groq_client = get_groq_client()

if groq_client:
    print("Successfully connected to Groq!")
else:
    print("Failed to connect to Groq. Please check your API key and network connection.")

Groq client initialized successfully.
Successfully connected to Groq!


## 🛠️ Step 3: Define Helper Functions

We need a few helper functions:
1.  `get_current_datetime`: An example function that the LLM can "call" if it needs the current time to answer a coding question.
2.  `filter_messages_for_api`: This function ensures we only send data to the API that it expects, removing any custom keys we might add for our own use (like evaluation results).

In [4]:
def get_current_datetime():
    """Returns the current date and time as a JSON string."""
    return json.dumps({"current_datetime": datetime.now().isoformat()})

def filter_messages_for_api(messages):
    """Removes custom keys like 'evaluation' before sending to API."""
    api_messages = []
    for msg in messages:
        api_msg = msg.copy()
        api_msg.pop("evaluation", None)
        api_messages.append(api_msg)
    return api_messages

print("Helper functions defined.")

Helper functions defined.


## 💬 Step 4: Interacting with the LLM - `run_conversation`

This is the core function that sends our messages to the LLM and gets a response.

In [5]:
def run_conversation(client, messages, model, tools=None, tool_choice="auto"):
    if not client:
        print("Groq client is not initialized. Cannot run conversation.")
        return None

    system_prompt = {
        "role": "system",
        "content": """You are a specialized Coding Assistant AI.
        Your primary function is to help users with programming and coding-related questions.
        Strictly adhere to the following rules:
        1. ONLY answer questions directly related to coding, programming languages, algorithms, data structures, software development tools, and concepts.
        2. If the user asks a question NOT related to coding, politely refuse to answer. State clearly that you can only assist with coding topics. Example refusal: "I specialize in coding-related questions. I cannot help with [topic of user's query]."
        3. Provide clear, concise, and accurate coding explanations or code snippets.
        4. If you need the current date or time to answer a coding-related question (e.g., about library versions released after a certain date), you can use the available 'get_current_datetime' function. Do not use it for non-coding purposes.
        5. Do not invent information. If you don't know the answer, say so.
        """,
    }
    conversation_history = [system_prompt] + messages

    try:
        response = client.chat.completions.create(
            model=model,
            messages=conversation_history,
            temperature=0.7,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response.choices[0].message
    except RateLimitError:
        print("Rate limit reached. Please wait and try again.")
        return None
    except APIError as e:
        print(f"Groq API Error: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during conversation: {e}")
        return None

print("`run_conversation` function defined.")

`run_conversation` function defined.


## 🔧 Step 5: Defining Tools for the LLM

LLMs can be given access to "tools" (functions) that they can use to get information or perform actions.

In [6]:
available_tools = {
    "get_current_datetime": get_current_datetime,
}

tools_schema = [
    {
        "type": "function",
        "function": {
            "name": "get_current_datetime",
            "description": "Get the current date and time as an ISO formatted string in JSON. Useful for time-sensitive coding questions. Only use if specifically needed for a coding context.",
            "parameters": {
                 "type": "object",
                 "properties": {},
                 "required": [],
            },
        },
    }
]

print("Tools schema and available tools defined.")

Tools schema and available tools defined.


## 📊 Step 6: Evaluating the Assistant's Response

This function will use another LLM call to assess the relevance and helpfulness of the assistant's response.

In [7]:
def evaluate_response(client, user_query, assistant_response_content, model):
    if not client:
        print("Groq client is not initialized. Cannot evaluate response.")
        return "Evaluation failed (client not initialized)."

    eval_system_prompt_content = f"""You are an evaluation AI. Evaluate the assistant's response based on the user's query.
    User Query: "{user_query}"
    Assistant Response: "{assistant_response_content}"

    Evaluate based on these criteria:
    1.  **Coding Relevance:** Was the assistant's response strictly related to coding/programming topics? (Yes/No)
    2.  **Helpfulness (if relevant):** If the response was coding-related, how helpful and accurate was it? (Score 1-5, 5=Excellent, 1=Not Helpful, NA if not relevant)
    3.  **Refusal Appropriateness (if irrelevant):** If the user's query was *not* coding-related, did the assistant politely refuse according to its instructions? (Yes/No/NA)

    Provide the evaluation concisely, starting with "Evaluation:".
    Example (Relevant): "Evaluation: Coding Relevance: Yes, Helpfulness: 4/5, Refusal Appropriateness: NA"
    Example (Irrelevant, Correct Refusal): "Evaluation: Coding Relevance: No, Helpfulness: NA, Refusal Appropriateness: Yes"
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "system", "content": eval_system_prompt_content}],
            temperature=0.1,
        )
        return response.choices[0].message.content
    except RateLimitError:
        print("Evaluation failed due to rate limit.")
        return "Evaluation failed (Rate Limit)."
    except APIError as e:
        print(f"Could not evaluate the response due to API error: {e}")
        return f"Evaluation failed (API Error: {e})."
    except Exception as e:
        print(f"Could not evaluate the response: {e}")
        return f"Evaluation failed ({type(e).__name__})."

print("`evaluate_response` function defined.")

`evaluate_response` function defined.


## 🗣️ Step 7: Let's Chat! - Interactive Mode

- Type your coding question and press Enter.
- Type `quit`, `exit`, or `bye` to end the chat.

In [ ]:
if not groq_client:
    print("Cannot start chat: Groq client is not initialized. Please check previous steps.")
else:
    print("\n--- Starting Interactive Chat with the Coding Assistant ---")
    print("Type 'quit', 'exit', or 'bye' to end the session.")
    print("-" * 50)

    chat_history = []

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit", "bye"]:
            print("\nAssistant: Goodbye! 👋")
            break

        if not user_input.strip():
            continue

        chat_history.append({"role": "user", "content": user_input})
        print("\nAssistant: 🧠 Thinking...")
        messages_for_api_call = filter_messages_for_api(chat_history)

        response_message = run_conversation(
            client=groq_client,
            messages=messages_for_api_call,
            model=DEFAULT_MODEL,
            tools=tools_schema,
            tool_choice="auto",
        )

        assistant_response_content = ""
        assistant_message_for_history = {}

        if response_message:
            if response_message.tool_calls:
                print("Assistant: 🛠️ Function call requested...")
                chat_history.append(response_message.model_dump(exclude_unset=True))
                tool_results_for_api = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    tool_call_id = tool_call.id
                    print(f"Assistant: Attempting to call function: {function_name}")
                    if function_name in available_tools:
                        function_to_call = available_tools[function_name]
                        try:
                            function_response_content = function_to_call()
                            tool_results_for_api.append(
                                {
                                    "tool_call_id": tool_call_id,
                                    "role": "tool",
                                    "name": function_name,
                                    "content": function_response_content,
                                }
                            )
                            print(f"Assistant: Function '{function_name}' executed.")
                        except Exception as e:
                            print(f"Error executing function {function_name}: {e}")
                            tool_results_for_api.append(
                                 {
                                    "tool_call_id": tool_call_id,
                                    "role": "tool",
                                    "name": function_name,
                                    "content": f"Error executing function: {e}",
                                }
                            )
                    else:
                         print(f"Assistant: Error - Function '{function_name}' not found.")
                         tool_results_for_api.append(
                            {
                                "tool_call_id": tool_call_id,
                                "role": "tool",
                                "name": function_name,
                                "content": f"Error: Function '{function_name}' is not defined.",
                            }
                        )
                chat_history.extend(tool_results_for_api)
                print("Assistant: ⚙️ Processing function results...")
                messages_for_api_call_2 = filter_messages_for_api(chat_history)
                final_response_message = run_conversation(
                    client=groq_client,
                    messages=messages_for_api_call_2,
                    model=DEFAULT_MODEL,
                    tool_choice="none",
                )
                if final_response_message and final_response_message.content:
                    assistant_response_content = final_response_message.content
                    assistant_message_for_history = final_response_message.model_dump(exclude_unset=True)
                else:
                    assistant_response_content = "Sorry, I encountered an error after processing the function call."
                    assistant_message_for_history = {"role": "assistant", "content": assistant_response_content}
            else:
                if response_message.content:
                    assistant_response_content = response_message.content
                    assistant_message_for_history = response_message.model_dump(exclude_unset=True)
                else:
                    assistant_response_content = "Sorry, I received an unexpected empty response."
                    assistant_message_for_history = {"role": "assistant", "content": assistant_response_content}
        else:
            assistant_response_content = "Sorry, I couldn't get a response due to an error."
            assistant_message_for_history = {"role": "assistant", "content": assistant_response_content}

        if assistant_message_for_history:
            chat_history.append(assistant_message_for_history)

        print(f"\nAssistant: {assistant_response_content}")

        if assistant_response_content and not assistant_response_content.startswith("Sorry"):
            print("\nAssistant: 📊 Evaluating response...")
            evaluation_result = evaluate_response(
                client=groq_client,
                user_query=user_input,
                assistant_response_content=assistant_response_content,
                model=EVALUATION_MODEL,
            )
            print(f"Assistant [Evaluation]: {evaluation_result}")
            if chat_history and chat_history[-1]["role"] == "assistant":
                 chat_history[-1]["evaluation"] = evaluation_result
        else:
            print("\nAssistant [Evaluation]: Skipped.")

        print("-" * 50)


--- Starting Interactive Chat with the Coding Assistant ---
Type 'quit', 'exit', or 'bye' to end the session.
--------------------------------------------------


## 🎉 Congratulations!

You've successfully set up and interacted with a Groq-powered Coding Assistant!

**Next Steps & Ideas:**
- Try different coding questions.
- Ask a non-coding question to see the refusal.
- Experiment with models and tools.